# Fetch Data Continuously

In [1]:
import requests
import time
from tqdm import tqdm
import pandas as pd
from datetime import datetime

def binance_recursive_fetch_2(coins, interval, starttime, endtime=None, data_type='spot'):
    """
    Fetch historical candlestick data from Binance API recursively for the specified coins and time range.
    
    Parameters:
    - coins (list): List of cryptocurrency symbols (e.g., ['BTC', 'ETH']).
    - interval (str): Candlestick interval (e.g., '1m', '5m', '1h').
    - starttime (int): Start time in milliseconds since epoch.
    - endtime (int, optional): End time in milliseconds since epoch. Defaults to the current time.
    - data_type (str, optional): 'spot' for spot data or 'futures' for futures data. Defaults to 'spot'.
    
    Returns:
    - dict: Contains the fetched data and the number of API calls per coin.
    """
    # Define the column structure
    BINANCE_CANDLE_COLUMNS = ['opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime',
                              'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

    if endtime is None:
        endtime = int(time.time() * 1000)  # Current time in milliseconds

    all_coins_result = {}
    data_list = []
    call_dict = {}

    for coin in tqdm(coins):
        result_list = []
        current_time = starttime
        call = 0
        timestamps = []

        while current_time < endtime:
            limit = min(1000, int((endtime - current_time) / (1000 * 60)) + 1)
            
            if data_type == 'spot':
                url = (f'https://api.binance.com/api/v3/klines'
                       f'?symbol={coin}USDT'
                       f'&startTime={current_time}'
                       f'&interval={interval}'
                       f'&limit={limit}')
            elif data_type == 'futures':
                url = (f'https://fapi.binance.com/fapi/v1/klines'
                       f'?symbol={coin}USDT'
                       f'&startTime={current_time}'
                       f'&interval={interval}'
                       f'&limit={limit}')
            else:
                raise ValueError("Invalid data_type. Choose either 'spot' or 'futures'.")

            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad HTTP responses
            result_list += response.json()

            if result_list:
                current_time = result_list[-1][0] + 60000  # Update to the next timestamp
                timestamps.append(current_time)
                call += 1

                if current_time >= endtime:
                    print(f"Reached endtime at {datetime.fromtimestamp(current_time / 1000).strftime('%Y-%m-%d %H:%M:%S')}. Stopping fetch.")
                    break

                print(f"{datetime.fromtimestamp(current_time / 1000).strftime('%Y-%m-%d %H:%M:%S')} "
                      f"status: {current_time < endtime}, time: {current_time}, calls: {call}")

            if len(timestamps) > 1 and timestamps[-1] == timestamps[-2]:
                print("Duplicate timestamp detected. Stopping fetch.")
                break

        current_df = pd.DataFrame(result_list, columns=BINANCE_CANDLE_COLUMNS)
        current_df['coin'] = coin
        current_df = current_df[['coin'] + BINANCE_CANDLE_COLUMNS]
        current_df = current_df.values.tolist()

        data_list += current_df
        call_dict.update({coin: call})

    return {'data': data_list, 'call': call_dict}

In [2]:
# Fetch the next data (increment by 1800000 ms, or 30 minutes)
data = binance_recursive_fetch_2(
    ['SOL'],
    '30m',
    starttime=int(1736339400000),
    endtime=int(1736463600000),
    data_type='futures'  # Fetch futures/sport
)

# Define the column names for the DataFrame based on the Binance API response structure
columns = ['coin', 'opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime', 
            'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

# Convert the list of data into a DataFrame
new_data = pd.DataFrame(data['data'], columns=columns)

# Drop unnecessary columns
new_data.drop(columns=['coin', 'volume', 'closetime', 'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused'], inplace=True)

# Convert columns 1-4 (openprice, highprice, lowprice, closeprice) to float
new_data[['openprice', 'highprice', 'lowprice', 'closeprice']] = new_data[['openprice', 'highprice', 'lowprice', 'closeprice']].apply(pd.to_numeric, errors='coerce')

new_data

100%|██████████| 1/1 [00:00<00:00,  9.61it/s]

Reached endtime at 2025-01-10 06:01:00. Stopping fetch.


,opentime,openprice,highprice,lowprice,closeprice
0,1736339400000,195.83,195.90,194.83,194.99
1,1736341200000,194.98,197.84,194.98,197.15
2,1736343000000,197.14,197.38,196.53,197.01
3,1736344800000,197.01,198.46,196.66,197.78
4,1736346600000,197.77,199.21,196.79,197.60
...,...,...,...,...,...
79,1736481600000,189.01,189.88,188.64,188.92
80,1736483400000,188.92,189.33,188.48,188.97
81,1736485200000,188.98,189.42,188.27,188.70
82,1736487000000,188.70,191.32,188.66,189.78


In [17]:
data

{'data': [['SOL',
   1736479800000,
   '188.3400',
   '189.3900',
   '187.9100',
   '189.0200',
   '147943',
   1736481599999,
   '27909311.5750',
   21774,
   '91091',
   '17182142.2150',
   '0'],
  ['SOL',
   1736481600000,
   '189.0100',
   '189.8800',
   '188.9000',
   '188.9500',
   '163538',
   1736483399999,
   '30968604.7820',
   22423,
   '80280',
   '15203305.6320',
   '0']],
 'call': {'SOL': 1}}

In [2]:
import pandas as pd
import time

def fetch_and_append_data():
    # Get the latest opentime from the CSV file
    current_df = pd.read_csv('sol_usdt_data.csv')
    last_opentime = current_df['opentime'].iloc[-1]

    # Print the last opentime and the length of the CSV before appending
    print(f"Last opentime in CSV: {last_opentime}")
    print(f"CSV length before appending: {len(current_df)}")

    # Get the current Unix timestamp in seconds
    current_timestamp = int(time.time())

    # Round down to the nearest 30 minutes (1800 seconds)
    rounded_timestamp = current_timestamp - (current_timestamp % 1800)

    # Convert the timestamp back to milliseconds
    rounded_timestamp_ms = rounded_timestamp * 1000

    # Print the current nearest previous rounded timestamp in ms
    print(f"Rounded timestamp (previous 30 minutes) in ms: {rounded_timestamp_ms}")

    # Initialize new_row_count to 0 by default
    new_row_count = 0

    # Check if there is new data to fetch (if rounded_timestamp_ms > last_opentime)
    if rounded_timestamp_ms > last_opentime:
        # Fetch the next data (increment by 1800000 ms, or 30 minutes)
        data = binance_recursive_fetch_2(
            ['SOL'],
            '30m',
            starttime=int(last_opentime + 1800000),
            endtime=int(last_opentime + 3600000),
            data_type='futures'  # Fetch futures/sport
        )

        # Define the column names for the DataFrame based on the Binance API response structure
        columns = ['coin', 'opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime', 
                   'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

        # Convert the list of data into a DataFrame
        new_data = pd.DataFrame(data['data'], columns=columns)

        # Drop unnecessary columns
        new_data.drop(columns=['coin', 'volume', 'closetime', 'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused'], inplace=True)

        # Convert columns 1-4 (openprice, highprice, lowprice, closeprice) to float
        new_data[['openprice', 'highprice', 'lowprice', 'closeprice']] = new_data[['openprice', 'highprice', 'lowprice', 'closeprice']].apply(pd.to_numeric, errors='coerce')

        # Check if there are new rows
        new_row_count = len(new_data)

        if new_row_count > 0:
            # Append the new data to the existing CSV
            new_data.to_csv('sol_usdt_data.csv', mode='a', header=False, index=False)
            
            # Print the number of new rows appended
            print(f"{new_row_count} new rows fetched and appended successfully.")
            # Print the new CSV length after appending
            current_df = pd.read_csv('sol_usdt_data.csv')
            print(f"New CSV length after appending: {len(current_df)}")
        else:
            print("No new data to append.")
    else:
        print("No new data available. The current timestamp is not greater than the last opentime.")

    return new_row_count

In [3]:
# Call the function to fetch and append the data
fetch_and_append_data()

Last opentime in CSV: 1736481600000
CSV length before appending: 441
Rounded timestamp (previous 30 minutes) in ms: 1736481600000
No new data available. The current timestamp is not greater than the last opentime.


0

# Slice the data

In [4]:
# Get the latest 51 data as the Ichimoku Cloud need 50 data
df_sliced = pd.read_csv('sol_usdt_data.csv').tail(52)
df_sliced

,opentime,openprice,highprice,lowprice,closeprice
389,1736389800000,196.27,197.21,195.30,197.03
390,1736391600000,197.04,197.23,194.80,194.95
391,1736393400000,194.95,195.31,194.06,194.28
392,1736395200000,194.28,194.54,194.02,194.15
393,1736397000000,194.42,194.90,194.42,194.52
394,1736398800000,195.29,195.37,195.21,195.21
395,1736400600000,194.98,195.16,194.36,194.53
396,1736402400000,194.22,194.31,194.07,194.08
397,1736404200000,194.36,194.52,194.36,194.46
398,1736406000000,193.31,193.31,193.04,193.08


# Add Super Trend Indicator

In [6]:
import pandas as pd
import numpy as np
from pandas_ta.overlap import hl2
from pandas_ta.volatility import atr
from pandas_ta.utils import get_offset, verify_series

def calculate_supertrend(df, high_col='highprice', low_col='lowprice', close_col='closeprice', length=10, multiplier=3.0, offset=0, drop_columns=True, **kwargs):
    """
    Calculate the Supertrend indicator and merge it with the original DataFrame.
    
    Parameters:
        df (DataFrame): Original DataFrame containing the OHLC data.
        high_col (str): Name of the high price column. Default is 'highprice'.
        low_col (str): Name of the low price column. Default is 'lowprice'.
        close_col (str): Name of the close price column. Default is 'closeprice'.
        length (int): Lookback period for ATR calculation. Default is 10.
        multiplier (float): Multiplier for ATR. Default is 3.0.
        offset (int): Number of periods to shift the result. Default is 0.
        drop_columns (bool): Whether to drop intermediate columns. Default is True.
        **kwargs: Additional arguments for handling NaN values (e.g., fillna).
    
    Returns:
        DataFrame: The original DataFrame with Supertrend columns merged.
    """
    # Validate Arguments
    high = verify_series(df[high_col], length)
    low = verify_series(df[low_col], length)
    close = verify_series(df[close_col], length)
    offset = get_offset(offset)

    if high is None or low is None or close is None:
        return df

    m = close.size
    dir_, trend = [1] * m, [0] * m
    long, short = [np.nan] * m, [np.nan] * m

    hl2_ = hl2(high, low)
    matr = multiplier * atr(high, low, close, length)
    upperband = hl2_ + matr
    lowerband = hl2_ - matr

    for i in range(1, m):
        if close.iloc[i] > upperband.iloc[i - 1]:
            dir_[i] = 1
        elif close.iloc[i] < lowerband.iloc[i - 1]:
            dir_[i] = -1
        else:
            dir_[i] = dir_[i - 1]
            if dir_[i] > 0 and lowerband.iloc[i] < lowerband.iloc[i - 1]:
                lowerband.iloc[i] = lowerband.iloc[i - 1]
            if dir_[i] < 0 and upperband.iloc[i] > upperband.iloc[i - 1]:
                upperband.iloc[i] = upperband.iloc[i - 1]

        if dir_[i] > 0:
            trend[i] = long[i] = lowerband.iloc[i]
        else:
            trend[i] = short[i] = upperband.iloc[i]

    _props = f"_{length}_{multiplier}"
    supertrend_df = pd.DataFrame({
        f"SUPERT{_props}": trend,
        f"SUPERTd{_props}": dir_,
        f"SUPERTl{_props}": long,
        f"SUPERTs{_props}": short,
    }, index=close.index)

    # Apply offset if needed
    if offset != 0:
        supertrend_df = supertrend_df.shift(offset)

    # Handle fills
    if "fillna" in kwargs:
        supertrend_df.fillna(kwargs["fillna"], inplace=True)
    if "fill_method" in kwargs:
        supertrend_df.fillna(method=kwargs["fill_method"], inplace=True)

    # Merge with original DataFrame
    result_df = df.join(supertrend_df)

    # Drop unnecessary columns
    if drop_columns:
        result_df.drop(columns=[f"SUPERT{_props}", f"SUPERTd{_props}"], inplace=True)

    return result_df

In [7]:
# Example usage
df_st = calculate_supertrend(df_sliced, length=10, multiplier=3.0)
df_st

,opentime,openprice,highprice,lowprice,closeprice,SUPERTl_10_3.0,SUPERTs_10_3.0
389,1736389800000,196.27,197.21,195.30,197.03,NaN,NaN
390,1736391600000,197.04,197.23,194.80,194.95,NaN,NaN
391,1736393400000,194.95,195.31,194.06,194.28,NaN,NaN
392,1736395200000,194.28,194.54,194.02,194.15,NaN,NaN
393,1736397000000,194.42,194.90,194.42,194.52,NaN,NaN
394,1736398800000,195.29,195.37,195.21,195.21,NaN,NaN
395,1736400600000,194.98,195.16,194.36,194.53,NaN,NaN
396,1736402400000,194.22,194.31,194.07,194.08,NaN,NaN
397,1736404200000,194.36,194.52,194.36,194.46,NaN,NaN
398,1736406000000,193.31,193.31,193.04,193.08,NaN,NaN


# Add Ichimoku Cloud Indicator

In [19]:
import pandas as pd
import numpy as np

def compute_ichimoku_with_supertrend(supertrend_df, conversion_periods=9, base_periods=26, span_b_periods=52, displacement=26):
    """
    Compute Ichimoku Cloud components and join them with the original supertrend DataFrame.
    
    Parameters:
    - supertrend_df (DataFrame): The input DataFrame with 'highprice', 'lowprice', and 'closeprice' columns.
    - conversion_periods (int): Period for the Conversion Line (Tenkan-sen). Default is 9.
    - base_periods (int): Period for the Base Line (Kijun-sen). Default is 26.
    - span_b_periods (int): Period for Leading Span B (Senkou Span B). Default is 52.
    - displacement (int): Displacement for Leading Spans. Default is 26.
    
    Returns:
    - DataFrame: A DataFrame that combines the original supertrend DataFrame with the computed Ichimoku Cloud components.
    """
    # Helper to calculate the average of the highest high and lowest low
    def donchian(data, period):
        return (data['highprice'].rolling(window=period).max() + 
                data['lowprice'].rolling(window=period).min()) / 2

    # Compute Ichimoku Cloud components
    supertrend_df['conversion_line'] = donchian(supertrend_df, conversion_periods)
    supertrend_df['base_line'] = donchian(supertrend_df, base_periods)
    supertrend_df['leading_span_a'] = ((supertrend_df['conversion_line'] + supertrend_df['base_line']) / 2).shift(displacement)
    supertrend_df['leading_span_b'] = donchian(supertrend_df, span_b_periods).shift(displacement)
    supertrend_df['lagging_span'] = supertrend_df['closeprice'].shift(-displacement)
    
    # Drop unnecessary columns
    supertrend_df.drop(columns=['conversion_line', 'base_line', 'lagging_span'], inplace=True)
    
    return supertrend_df

In [20]:
# Usage example:
df_st_ic = compute_ichimoku_with_supertrend(df_st)
df_st_ic

,opentime,openprice,highprice,lowprice,closeprice,SUPERTl_10_3.0,SUPERTs_10_3.0,leading_span_a,leading_span_b
302,1736233200000,216.23,216.62,215.87,215.91,NaN,NaN,NaN,NaN
303,1736235000000,215.92,215.98,215.33,215.97,NaN,NaN,NaN,NaN
304,1736236800000,215.96,216.05,215.27,215.70,NaN,NaN,NaN,NaN
305,1736238600000,215.71,216.79,215.66,216.50,NaN,NaN,NaN,NaN
306,1736240400000,216.51,216.62,215.75,215.82,NaN,NaN,NaN,NaN
307,1736242200000,215.82,215.95,214.67,214.98,NaN,NaN,NaN,NaN
308,1736244000000,214.98,214.98,214.21,214.95,NaN,NaN,NaN,NaN
309,1736245800000,214.95,215.11,214.61,214.76,NaN,NaN,NaN,NaN
310,1736247600000,214.75,215.33,214.65,215.33,NaN,NaN,NaN,NaN
311,1736249400000,215.33,215.33,212.73,213.95,NaN,NaN,NaN,NaN


# Define Action Suggestion

In [28]:
# Set default
prev_action = None

import pandas as pd

def determine_suggested_action(df):
    """
    Determines the suggested action (Long, Short, or None) based on the Supertrend and Ichimoku values.

    Parameters:
        df (DataFrame): The DataFrame containing the necessary columns:
                        'SUPERTl_10_3.0', 'SUPERTs_10_3.0',
                        'leading_span_a', 'leading_span_b', and 'closeprice'.

    Returns:
        str: Suggested action ('Long', 'Short', or None').
    """
    # Get the last row of the DataFrame
    last_row = df.tail(1).copy()

    # Rename the last 4 columns for convenience
    new_column_names = {
        'SUPERTl_10_3.0': 'Up Trend',
        'SUPERTs_10_3.0': 'Down Trend',
        'leading_span_a': 'Leading Span A',
        'leading_span_b': 'Leading Span B'
    }
    last_row = last_row.rename(columns=new_column_names)

    # Extract scalar values
    up_trend = last_row['Up Trend'].iloc[0]
    down_trend = last_row['Down Trend'].iloc[0]
    closeprice = last_row['closeprice'].iloc[0]
    leading_span_a = last_row['Leading Span A'].iloc[0]
    leading_span_b = last_row['Leading Span B'].iloc[0]

    # Determine the suggested action
    if pd.notna(up_trend) and closeprice > leading_span_a and closeprice > leading_span_b:
        return 'Long'
    elif pd.notna(down_trend) and closeprice < leading_span_a and closeprice < leading_span_b:
        return 'Short'
    else:
        return None

In [30]:
# Example usage
suggested_action = determine_suggested_action(df_st_ic)
print(f"Suggested Action: {suggested_action}")

Suggested Action: None


# Binance API Class

In [1]:
import requests
import hashlib
import urllib.parse
import hmac
import time

class BinanceAPI:
    def __init__(self, api_key: str, api_secret: str, testnet: bool = False):
        self.api_key = api_key  
        self.api_secret = api_secret
        if testnet:
            self.base_url = "https://testnet.binancefuture.com"
        else:
            self.base_url = "https://fapi.binance.com"

    def _generate_signature(self, params: dict) -> str:
        """Generate HMAC SHA256 signature for request"""
        query_string = urllib.parse.urlencode(params)
        return hmac.new(
            self.api_secret.encode("utf-8"),
            query_string.encode("utf-8"),
            hashlib.sha256,
        ).hexdigest()

    def _send_request(
        self, method: str, endpoint: str, params: dict = None, signed: bool = False
    ) -> dict:
        """Send request to Binance FAPI"""
        url = f"{self.base_url}{endpoint}"
        headers = {"X-MBX-APIKEY": self.api_key}

        if params is None:
            params = {}

        if signed:
            params["timestamp"] = int(time.time() * 1000)
            params["signature"] = self._generate_signature(params)

        try:
            if method == "GET":
                response = requests.get(url, headers=headers, params=params)
            elif method == "POST":
                response = requests.post(url, headers=headers, params=params)
            elif method == "DELETE":
                response = requests.delete(url, headers=headers, params=params)
            else:
                raise ValueError(f"Invalid method: {method}")

            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            logging.error(f"API request failed: {e}")
            return None

    def get_mark_price(self, symbol: str) -> dict:
        """Get mark price for symbol"""
        params = {"symbol": symbol}
        return self._send_request("GET", "/fapi/v1/premiumIndex", params)

    def get_position_risk(self, symbol: str = None) -> dict:
        """Get position information"""
        params = {}
        if symbol:
            params["symbol"] = symbol
        return self._send_request("GET", "/fapi/v3/positionRisk", params, signed=True)

    def create_order(
        self,
        symbol: str,
        side: str,
        order_type: str,
        quantity: float,
        price: float = None,
        reduce_only: bool = False,
    ) -> dict:
        """Create a new order"""
        params = {
            "symbol": symbol,
            "side": side,
            "type": order_type,
            "quantity": quantity,
            "reduceOnly": reduce_only,
        }

        if price:
            params["price"] = price

        return self._send_request("POST", "/fapi/v1/order", params, signed=True)
    
    def close_all_position(self):
        """Close all active positions on the account"""
        # Get all active positions
        positions = self.get_position_risk()
        if positions is None:
            logging.error("Failed to retrieve position information.")
            return

        # Iterate through each position and close it
        for position in positions:
            symbol = position['symbol']
            position_amt = float(position['positionAmt'])

            if position_amt != 0:
                side = 'SELL' if position_amt > 0 else 'BUY'
                quantity = abs(position_amt)

                # Create a market order to close the position
                response = self.create_order(
                    symbol=symbol,
                    side=side,
                    order_type="MARKET",
                    quantity=quantity,
                    reduce_only=True
                )

                if response:
                    logging.info(f"Closed position: {symbol} with quantity {quantity}")
                else:
                    logging.error(f"Failed to close position: {symbol}")
                    
    def ping(self) -> dict:
        """Ping the Binance API to check connectivity."""
        return self._send_request("GET", "/fapi/v1/ping")
    
    def get_server_time(self) -> dict:
        """Get Binance server time to confirm connectivity."""
        return self._send_request("GET", "/fapi/v1/time")
    
    def check_balance(self):
        endpoint = "/fapi/v2/balance"
        params = {
            "timestamp": int(time.time() * 1000)
        }
        params["signature"] = self._generate_signature(params)
        return self._send_request("GET", endpoint, params=params)

In [2]:
# Replace these with your testnet credentials
API_KEY = "TGZ6PvNeQc3c3ctlzm0UOdkgr1fi5oEMMPXDK9Dns51VXGKYGIirlOJ8de5TYNRC"
API_SECRET = "Ng4YmUDDzq7W9l5F08qcY3Qq2OXms4xE7A9nlslDIxP2agjVWqmZbOOxCRTZEHOl"

# Instantiate BinanceAPI for Testnet
binance_api = BinanceAPI(api_key=API_KEY, api_secret=API_SECRET, testnet=False)

In [3]:
# Check connectivity with ping
ping_response = binance_api.ping()
print("Ping Response:", ping_response)  # Should return an empty dictionary if successful

# Check server time
server_time = binance_api.get_server_time()
print("Server Time:", server_time)  # Should return the server time in milliseconds

Ping Response: {}
Server Time: {'serverTime': 1736826911597}


In [4]:
# Get balance information
balance = binance_api.check_balance()

# Print the balance information
if balance:
    print("Balance Info:", balance)

NameError: name 'logging' is not defined

In [6]:
# Initialize connection to Binance
API_KEY = "TGZ6PvNeQc3c3ctlzm0UOdkgr1fi5oEMMPXDK9Dns51VXGKYGIirlOJ8de5TYNRC"
API_SECRET = "Ng4YmUDDzq7W9l5F08qcY3Qq2OXms4xE7A9nlslDIxP2agjVWqmZbOOxCRTZEHOl"
binance_api = BinanceAPI(api_key=API_KEY, api_secret=API_SECRET, testnet=False)

symbol = 'SOLUSDT'

gpr = binance_api.get_position_risk(symbol=symbol)
if float(gpr[0]['entryPrice']) < float(gpr[0]['entryPrice']):
    prev_action = 'Long'
elif float(gpr[0]['entryPrice']) > float(gpr[0]['entryPrice']):
    prev_action = 'Short'
else:
    prev_action = None

prev_action   

In [84]:
# long_order_response = binance_api.create_order(
#     symbol="SUIUSDT",    # Trading pair
#     side="BUY",          # Buy to open a long position
#     order_type="MARKET", # Market order for instant execution
#     quantity=2         # Valid quantity in SUI (base asset)
# )
# print("Long Order Response:", long_order_response)

Long Order Response: {'orderId': 15823060022, 'symbol': 'SUIUSDT', 'status': 'NEW', 'clientOrderId': 'Nib5vNrhppQas42ffdgyUO', 'price': '0.000000', 'avgPrice': '0.00', 'origQty': '2.0', 'executedQty': '0.0', 'cumQty': '0.0', 'cumQuote': '0.0000000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.000000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1736412266462}


In [83]:
# long_order_response = binance_api.create_order(
#     symbol="SUIUSDT",    # Trading pair
#     side="SELL",          # Buy to open a long position
#     order_type="MARKET", # Market order for instant execution
#     quantity=2         # Valid quantity in SUI (base asset)
# )
# print("Short Order Response:", long_order_response)

Short Order Response: {'orderId': 15822875814, 'symbol': 'SUIUSDT', 'status': 'NEW', 'clientOrderId': 'utbUXsoju4AtvJ2daK0we6', 'price': '0.000000', 'avgPrice': '0.00', 'origQty': '2.0', 'executedQty': '0.0', 'cumQty': '0.0', 'cumQuote': '0.0000000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.000000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1736412121900}


In [43]:
import math

get_initial_price = binance_api.get_mark_price(symbol='SOLUSDT')
print("Initial Mark Price:", get_initial_price)

trade_ammount_usdt = 1000
initial_mark_price = float(get_initial_price['markPrice'])  # Convert to float
quantity = trade_ammount_usdt / initial_mark_price
quantity = math.floor(quantity * 1000) / 1000

print("Quantity:", quantity)

get_last_price = binance_api.get_mark_price(symbol='SOLUSDT')
last_mark_price = float(get_last_price['markPrice'])  # Convert to float
print("Last Mark Price:", last_mark_price)

print("Last Quantity:", quantity*last_mark_price)

Initial Mark Price: {'symbol': 'SOLUSDT', 'markPrice': '183.94000000', 'indexPrice': '184.03719791', 'estimatedSettlePrice': '184.09272903', 'lastFundingRate': '-0.00003037', 'interestRate': '0.00010000', 'nextFundingTime': 1736841600000, 'time': 1736825796000}
Quantity: 5.436
Last Mark Price: 183.94
Last Quantity: 999.89784


In [48]:
long_order_response = binance_api.create_order(
    symbol="SOLUSDT",    # Trading pair
    side="BUY",          # Buy to open a long position
    order_type="MARKET", # Market order for instant execution
    quantity= 1
)
print("Order Response:", long_order_response)

Order Response: {'orderId': 91899502171, 'symbol': 'SOLUSDT', 'status': 'NEW', 'clientOrderId': 'sHA92MLfcvwZJfOXcizFob', 'price': '0.0000', 'avgPrice': '0.00', 'origQty': '1', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.0000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.0000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1736826002341}


In [27]:
type(quantity)

float

In [5]:
import math

# Example values
trade_amount_usdt = 1000
mark_price = 180.18  # Replace with actual mark price
step_size = 0.01  # Replace with the correct step size from Binance's exchangeInfo

# Calculate quantity
raw_quantity = trade_amount_usdt / mark_price
quantity = math.floor(raw_quantity / step_size) * step_size

print("Adjusted Quantity:", quantity)  # This should now be valid for Binance

Adjusted Quantity: 5.55


In [16]:
gpr = binance_api.get_position_risk(symbol='SOLUSDT')
print(gpr)

[{'symbol': 'SOLUSDT', 'positionSide': 'BOTH', 'positionAmt': '5', 'entryPrice': '183.15', 'breakEvenPrice': '183.241575', 'markPrice': '183.44000000', 'unRealizedProfit': '1.45000000', 'liquidationPrice': '0', 'isolatedMargin': '0', 'notional': '917.20000000', 'marginAsset': 'USDT', 'isolatedWallet': '0', 'initialMargin': '917.20000000', 'maintMargin': '4.58600000', 'positionInitialMargin': '917.20000000', 'openOrderInitialMargin': '0', 'adl': 2, 'bidNotional': '0', 'askNotional': '0', 'updateTime': 1736811155193}]


In [19]:
# Identify current position
gpr = binance_api.get_position_risk(symbol='SOLUSDT')
if float(gpr[0]['entryPrice']) < float(gpr[0]['entryPrice'])
    prev_action = 'Long'
elif float(gpr[0]['entryPrice']) > float(gpr[0]['entryPrice'])
    prev_action = 'Short'
else
    prev_action = None


183.15

In [37]:
float(gpr[0]['positionAmt'])

5.0

# Action Real Action

In [32]:
def handle_trading_action(suggested_action, prev_action=prev_action):
    """
    Handles trading actions based on the suggested action.

    Parameters:
        suggested_action (str): The suggested action ('Long', 'Short', or None).
        prev_action (str, optional): The previous action. Defaults to None.

    Returns:
        tuple: A tuple containing the current action and the updated previous action.
    """
    print(f"Previous Action: {prev_action}")
    print(f"Suggested Action: {suggested_action}")
    
    # Initialize current action
    curr_action = None
    
    # Action handling logic
    if prev_action == suggested_action:
        pass  # Do nothing if the action is the same
    else:
        if prev_action is None and suggested_action == 'Long':
            curr_action = 'Open Long'
            print(f'Open a Long Position: {curr_action}')
            prev_action = 'Long'
        elif prev_action is None and suggested_action == 'Short':
            curr_action = 'Open Short'
            print(f'Open a Short Position: {curr_action}')
            prev_action = 'Short'
        elif prev_action == 'Long' and suggested_action == 'Short':
            curr_action = 'Close Long & Open Short'
            print(f'Close all positions: {curr_action}')
            prev_action = 'Short'
        elif prev_action == 'Long' and suggested_action is None:
            pass  # Do nothing
        elif prev_action == 'Short' and suggested_action == 'Long':
            curr_action = 'Close Short & Open Long'
            print(f'Close all positions: {curr_action}')
            prev_action = 'Long'
        elif prev_action == 'Short' and suggested_action is None:
            pass  # Do nothing

    # Print the result
    print(f"Current Action: {curr_action if curr_action else 'No action taken'}")
    print(f"New Previous Action: {prev_action}")
    
    return curr_action, prev_action

In [33]:
current_action, new_prev_action = handle_trading_action(suggested_action, prev_action)

Previous Action: None
Suggested Action: None
Current Action: No action taken
New Previous Action: None


# Fixing Data Fetch

In [47]:
def binance_recursive_fetch_2(coins, interval, starttime, endtime=None, data_type='futures'):

    # Define the column structure
    BINANCE_CANDLE_COLUMNS = ['opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime',
                              'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

    if endtime is None:
        endtime = int(time.time() * 1000)  # Current time in milliseconds

    all_coins_result = {}
    data_list = []
    call_dict = {}

    for coin in tqdm(coins):
        result_list = []
        current_time = starttime
        call = 0
        timestamps = []

        while current_time < endtime:
            limit = min(1000, int((endtime - current_time) / (1000 * 60)) + 1)
            
            if data_type == 'spot':
                url = (f'https://api.binance.com/api/v3/klines'
                       f'?symbol={coin}USDT'
                       f'&startTime={current_time}'
                       f'&interval={interval}'
                       f'&limit={limit}')
            elif data_type == 'futures':
                url = (f'https://fapi.binance.com/fapi/v1/klines'
                       f'?symbol={coin}USDT'
                       f'&startTime={current_time}'
                       f'&interval={interval}'
                       f'&limit={limit}')
            else:
                raise ValueError("Invalid data_type. Choose either 'spot' or 'futures'.")

            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad HTTP responses
            result_list += response.json()

            if result_list:
                current_time = result_list[-1][0] + 60000  # Update to the next timestamp
                timestamps.append(current_time)
                call += 1

                if current_time >= endtime:
                    # logging.info(f"Reached endtime at {datetime.fromtimestamp(current_time / 1000).strftime('%Y-%m-%d %H:%M:%S')}. Stopping fetch.")
                    break

                # logging.info(f"{datetime.fromtimestamp(current_time / 1000).strftime('%Y-%m-%d %H:%M:%S')} "
                    #   f"status: {current_time < endtime}, time: {current_time}, calls: {call}")

            if len(timestamps) > 1 and timestamps[-1] == timestamps[-2]:
                # logging.info("Duplicate timestamp detected. Stopping fetch.")
                break

        current_df = pd.DataFrame(result_list, columns=BINANCE_CANDLE_COLUMNS)
        current_df['coin'] = coin
        current_df = current_df[['coin'] + BINANCE_CANDLE_COLUMNS]
        current_df = current_df.values.tolist()

        data_list += current_df
        call_dict.update({coin: call})

    return {'data': data_list, 'call': call_dict}

In [39]:
import time
from datetime import datetime, timedelta

# Get the current time
now = datetime.now()

# Set the start time to 10:00 AM today
start_time = datetime(now.year, now.month, now.day, 2, 10, 0)

# Set the end time to 12:30 PM today
end_time = datetime(now.year, now.month, now.day, 2, 30, 0)

# Convert to milliseconds (as expected by the Binance API)
start_time = int(start_time.timestamp() * 1000)
end_time = int(end_time.timestamp() * 1000)

# Fetch data
data = binance_recursive_fetch_2(
    ['SOL'],
    '30m',
    starttime=start_time,
    data_type='futures'  # Fetch futures data
)

# Define the column names for the DataFrame based on the Binance API response structure
columns = ['coin', 'opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime',
            'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

# Convert the list of data into a DataFrame
new_data = pd.DataFrame(data['data'], columns=columns)

# Drop unnecessary columns
new_data.drop(columns=['coin', 'volume', 'closetime', 'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused'], inplace=True)

# Convert columns 1-4 (openprice, highprice, lowprice, closeprice) to float
new_data[['openprice', 'highprice', 'lowprice', 'closeprice']] = new_data[['openprice', 'highprice', 'lowprice', 'closeprice']].apply(pd.to_numeric, errors='coerce')

# Check if there are new rows
new_data = new_data.iloc[:-1]

new_data


100%|██████████| 1/1 [00:00<00:00,  3.09it/s]


,opentime,openprice,highprice,lowprice,closeprice
0,1736476200000,188.81,189.30,188.03,188.18
1,1736478000000,188.18,188.72,187.86,188.35
2,1736479800000,188.34,189.39,187.91,189.02
3,1736481600000,189.01,189.88,188.64,188.92


In [40]:
current_df = pd.read_csv('/home/ubuntu/Rheza/local-share/03X_ST_IC/02_prod/sol_usdt_data.csv')
last_opentime = current_df['opentime'].iloc[-1]
print(f"Last opentime in CSV: {last_opentime}")

Last opentime in CSV: 1736479800000


In [41]:
# Get the current Unix timestamp in seconds
current_timestamp = int(time.time())

# Round down to the nearest 30 minutes (1800 seconds)
rounded_timestamp = current_timestamp - (current_timestamp % 1800)

# Convert the timestamp back to milliseconds
rounded_timestamp_ms = rounded_timestamp * 1000

# Log the rounded timestamp for comparison
print(f"Rounded timestamp: {rounded_timestamp_ms}, Last opentime in CSV: {last_opentime}")

Rounded timestamp: 1736483400000, Last opentime in CSV: 1736479800000


In [65]:
def fetch_and_append_data():
    # Get the latest opentime from the CSV file
    try:
        current_df = pd.read_csv('/home/ubuntu/Rheza/local-share/03X_ST_IC/02_prod/sol_usdt_data.csv')
        last_opentime = current_df['opentime'].iloc[-1]
    except FileNotFoundError:
        logging.error("CSV file not found. Ensure the path is correct.")
        return 0
    except Exception as e:
        logging.error(f"Error reading CSV file: {e}")
        return 0

    # Log the last opentime and the length of the CSV before appending
    logging.info(f"Last opentime in CSV: {last_opentime}")
    logging.info(f"CSV length before appending: {len(current_df)}")

    # Get the current Unix timestamp in seconds
    current_timestamp = int(time.time())

    # Round down to the nearest 30 minutes (1800 seconds)
    rounded_timestamp = current_timestamp - (current_timestamp % 1800)

    # Convert the timestamp back to milliseconds
    rounded_timestamp_ms = rounded_timestamp * 1000

    # Log the rounded timestamp for comparison
    logging.info(f"Rounded timestamp: {rounded_timestamp_ms}, Last opentime in CSV: {last_opentime}")

    # Initialize new_row_count to 0 by default
    new_row_count = 0

    # Check if there is new data to fetch (if rounded_timestamp_ms > last_opentime)
    if rounded_timestamp_ms > last_opentime:
        # Fetch the next data (increment by 1800000 ms, or 30 minutes)
        try:
            data = binance_recursive_fetch_2(
                ['SOL'],
                '30m',
                starttime=int(last_opentime + 1800000),
                endtime=None,
                data_type='futures'  # Fetch futures/sport
            )

            print(f"Fetched data: {data}")
            # logging.info(f"Fetched data: {data}")

            # Check if the data exists
            if 'data' in data and data['data']:
                # Define the column names for the DataFrame based on the Binance API response structure
                columns = ['coin', 'opentime', 'openprice', 'highprice', 'lowprice', 'closeprice', 'volume', 'closetime',
                           'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused']

                # Convert the list of data into a DataFrame
                new_data = pd.DataFrame(data['data'], columns=columns)

                # Drop unnecessary columns
                new_data.drop(columns=['coin', 'volume', 'closetime', 'quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'unused'], inplace=True)

                # Convert columns 1-4 (openprice, highprice, lowprice, closeprice) to float
                new_data[['openprice', 'highprice', 'lowprice', 'closeprice']] = new_data[['openprice', 'highprice', 'lowprice', 'closeprice']].apply(pd.to_numeric, errors='coerce')
                
                # Check if there are new rows
                new_data = new_data.iloc[:-1]

                # Check if there are new rows
                new_row_count = len(new_data)

                if new_row_count > 0:
                    # Append the new data to the existing CSV
                    new_data.to_csv('/home/ubuntu/Rheza/local-share/03X_ST_IC/02_prod/sol_usdt_data.csv', mode='a', header=False, index=False)
                    
                    # Log the number of new rows appended
                    logging.info(f"{new_row_count} new rows fetched and appended successfully.")

                    # Log the new CSV length after appending
                    current_df = pd.read_csv('/home/ubuntu/Rheza/local-share/03X_ST_IC/02_prod/sol_usdt_data.csv')
                    logging.info(f"New CSV length after appending: {len(current_df)}")
                    logging.info(f"New data: {current_df.tail(1)}")
                else:
                    logging.info("No new data to append.")
            else:
                logging.info("No new data fetched from Binance.")
        except Exception as e:
            logging.error(f"Error fetching data from Binance: {e}")
    else:
        logging.info("No new data available. The current timestamp is not greater than the last opentime.")

    return new_row_count

In [68]:
fetch_and_append_data()

Last opentime in CSV: 1736483400000
CSV length before appending: 442
Rounded timestamp: 1736485200000, Last opentime in CSV: 1736483400000


100%|██████████| 1/1 [00:00<00:00,  5.05it/s]

Fetched data: {'data': [['SOL', 1736485200000, '188.9800', '189.1800', '188.7900', '188.9100', '26683', 1736486999999, '5042688.6560', 4755, '11450', '2163997.7260', '0']], 'call': {'SOL': 2}}
No new data to append.


0